# Постановка задачи

![](https://obraz-asino.ru/media/moderator/Archive2015/50/c4et4ik.jpg)

В энергосбытовой компании (далее - ЭСК) есть несознательные абоненты, которые имеют просроченную задолженность за потребленную электроэнергию. С этими должниками проводится некоторая работа. По результатам работы от должников поступает оплата. Каждый день в ЭСК через банки, почтовые переводы, онлайн-сервисы и кассы компании регистрируются деньги от абонентов, в том числе и от должников. Необходимо предсказать, каковы будут оплаты от должников в ближайший год в целом по всем подразделениям ЭСК, расположенных в разных регионах РФ.    
     
Для предсказания из рабочей базы данных ЭСК получена информация о ежедневнем поступлении платежей от абонентов, числившихся в момент оплаты должниками с просрочкой долга. Абоненты, чьи неоплаченные счета ещё не были признаны просроченными, в выборку для предсказания не включались. Выборка представляет из себя временной ряд с шагом в 1 день.    
     
Выборка содержит суммы высокого порядка (сотни тысяч и миллионы рублей), поскольку информация составлена суммарно по нескольким регионам РФ. Суммы большие еще и по той причине, что должниками признаются абоненты, у которых просрочка оплаты составляет 2 месяца и более, объемом в 2 норматива потребления и более. Столь низкий порог для признания абонента должником помещает не менее четверти всех абонентов в разряд должников.
    
Предсказание оплаты долга позволит ЭСК корректнее планировать бюджет и расходы.

**Предупреждение:** данные в процессе сбора были изменены на некоторый коэффициент. Поэтому приводимые здесь суммы не могут раскрывать коммерческую тайну о финансовом положении ЭСК.

# Библиотеки и функции

In [ ]:
import numpy as np 
from numpy import array
import pandas as pd 
from datetime import datetime, date, timedelta 
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import statsmodels.api as sm 
import math as m
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, RobustScaler, MaxAbsScaler

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.statespace.sarimax import SARIMAXResults

from statsmodels.graphics.tsaplots import plot_acf,plot_pacf 
from statsmodels.tsa.seasonal import seasonal_decompose 
                   
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tools.eval_measures import rmse

import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import keras
from keras.layers import Dense, Input, LSTM, Dropout, Activation, SimpleRNN
from keras.layers import Bidirectional, Flatten, RepeatVector
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers, Sequential
from keras.models import load_model

from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

from fbprophet import Prophet
import holidays
import json
from fbprophet.serialize import model_to_json, model_from_json
import pickle

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# Глобальные константы
START_DATE = "2020-01-31" # Дата отсечки тренировочных данных от тестовых
PRED_LEN = 446            # Длина тестовых данных
PRED_COLUMN = "PAY"       # Имя признака во временном ряду
INPUT_LEN = 30            # Длина скользящего окна (месяц)

In [ ]:
# Mean absolute percentage_error
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# Функция для построения графика истинного ряда и его предсказания
def plot_prediction(df, prediction): 
    plt.figure(figsize=(25, 12))
    plt.plot(df.index[df.index>=START_DATE], prediction, label="prediction", alpha=.7)
    plt.plot(df.index[df.index>=START_DATE], df.loc[df.index>=START_DATE, PRED_COLUMN], label="real", alpha=.7)
    plt.scatter(df.index[df.index>=START_DATE], prediction, label="prediction", alpha=.7)
    plt.scatter(df.index[df.index>=START_DATE], df.loc[df.index>=START_DATE, PRED_COLUMN], label="real", alpha=.7)
    plt.legend()
    plt.title(PRED_COLUMN + " Prediction")
    plt.show()

# Чтение, преобразование и анализ данных

### Данные временного ряда

In [ ]:
# Читаем данные по оплатам должников, выгруженные из рабочей БД
df = pd.read_csv('../input/payments/pay.csv', sep=';')

# Смотрим на данные
display(df.head(5))
display(df.tail(5))

**Пояснение по столбцам:**    
Первый столбец без имени - порядковый номер строки. Столбец создан автоматически при экспорте данных в файл.    
PAY - сумма оплат по задолженностям, поступившая в текущий день     
CNT - количество оплат по задолженностям, поступившее в текущий день    
PAY_DATE - текущий день оплаты

In [ ]:
# Последние 2 строки явно выбиваются из общего ряда. Удалим их из датасета
df = df.iloc[:-2]

In [ ]:
# Информация о наборе данных
display(df.info())

In [ ]:
# Столбец PAY имеет строковый вид. Преобразуем его в число
df['PAY'] = df['PAY'].apply(lambda x: float(x.replace(',','.')))

In [ ]:
# Проверим статистики числовых столбцов
display(df.describe())

Предметом анализа будет являться только признак PAY. Безымянный столбец и столбец с количеством удалим, а столбец с оплатой разделим на 1000, чтобы оперировать меньшими суммами.    
Строковые даты превратим в истинные даты и столбец с ними сделаем индексом.

In [ ]:
# Переходим к тысячам рублей
df['PAY'] = df['PAY']/1000

# Строковый столбец с датой оплаты преобразуем к дате и сортируем по дате
df['Date'] = pd.to_datetime(df['PAY_DATE'], format = '%d.%m.%Y')
df = df.sort_values(by='Date')

# Превращаем дату в индекс
df = df.set_index(pd.DatetimeIndex(df['Date']))

# Удаляем ненужные столбцы
df = df.drop(df.columns[0], axis=1)
df = df.drop(['Date', 'PAY_DATE', 'CNT'], axis =1)
df

In [ ]:
# Проверим, есть ли пропуски в датах
print(len(df.index))
print((df.index[-1]-df.index[0]).days + 1)

Пропусков по дням нет. Интерполяция данных не требуется.

In [ ]:
# Оформим в функцию чтение и обработку датасета
def get_data():
    df = pd.read_csv('../input/payments/pay.csv', sep=';')
    df = df.iloc[:-2]
    df['PAY'] = df['PAY'].apply(lambda x: float(x.replace(',','.')))
    df['PAY'] = df['PAY']/1000
    df['Date'] = pd.to_datetime(df['PAY_DATE'], format = '%d.%m.%Y')
    df = df.sort_values(by='Date')
    df = df.set_index(pd.DatetimeIndex(df['Date']))
    df = df.drop(df.columns[0], axis=1)
    df = df.drop(['Date', 'PAY_DATE', 'CNT'], axis =1)
    
    return df

In [ ]:
# Сохраним последний индекс набора данных для отсечки реальных и спрогнозированных данных
LAST_REAL = df.index[-1]
LAST_REAL

### Визуализация временного ряда

In [ ]:
# Построим график зависимости сумм оплат от времени
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df.PAY, mode='lines'))
fig.show() 

По графику видно, что выделяются пики оплат и что оплаты носят колебательный (сезонный характер). При этом пик оплат до 2014 года приходился на декабрь, а начиная с 2014 - на февраль.Скорее всего, это связано с тем, до 2013 года расчет за декабрь и премии выплачивались до наступления нового года, а начиная с 2014 года и с учетом кризиса в экономике - после новогодних каникул.    
Также видно, что амплитуда колебаний растёт. связано это с тем, что тарифы на электроэнергию с годами росли, соответственно, росли и оплаты.     
     
Для наглядности построим укрупненный график за 2 года, год, полгода, месяц.

In [ ]:
# Укрупненные выборочные графики оплат
f, ax = plt.subplots(4, 1, figsize=(20, 20))
ax[0].plot(df.PAY[(df.index >= '2015-05-01') & (df.index <= '2017-05-01')], c='g')
ax[0].set_title('Данные за 2 года')
ax[0].set_xlabel('Дата')
ax[0].set_ylabel('Сумма')

ax[1].plot(df.PAY[(df.index >= '2016-05-01') & (df.index <= '2017-05-01')], c='g')
ax[1].set_title('Данные за год')
ax[1].set_xlabel('Дата')
ax[1].set_ylabel('Сумма')

ax[2].plot(df.PAY[(df.index >= '2016-11-01') & (df.index <= '2017-05-01')], c='g')
ax[2].set_title('Данные за полгода')
ax[2].set_xlabel('Дата')
ax[2].set_ylabel('Сумма')

ax[3].plot(df.PAY[(df.index >= '2017-04-01') & (df.index <= '2017-05-01')], c='g')
ax[3].set_title('Данные за месяц')
ax[3].set_xlabel('Дата')
ax[3].set_ylabel('Сумма')

plt.show()

Колебания в оплатах идут:    
* по годам,
* по месяцам в году,
* по неделям в месяцах,
* по дням в неделях.

In [ ]:
# Проверим распределение исследуемой величины
df.PAY.hist(bins=100)

Распределение величины - сумма двух почти нормальных распределений. Если удалить из рассмотрения все оплаты  суммой < примерно 6000, то модели классического ML при генерации новых дополнительных признаков могли бы показать неплохие результаты. Но таких платежей достаточно много, поэтому удалять ничего не будем.

In [ ]:
# Визуализируем скользящее среднее
roll_mean = df.PAY.rolling(window = 100).mean()
plt.figure(figsize=(20,8))
plt.plot(df.PAY,label="Актуальные значения")
plt.plot(roll_mean,label="Скользящее среднее")
plt.legend(loc="upper left")
plt.show()

График показывает, что сезонные колебания - годовые и месячные - есть даже у сглаженного ряда, составленного из скользящих средних.    
      
Проверим сезонность и тренд, разложив временной ряд на составляющие. Сезонную декомпозицию проверим в двух вариантах - мультипликативном и аддитивном. Для исследуемого ряда более подходит мультипликативное разложение, так как со временем увеличиваются не только средние значения, но и амплитуды колебаний. Тем не менеее, проверим оба варианта.

In [ ]:
# Применяем разложение на тренд, сезонность и остаток для мультипликативной модели 
#    (временной ряд = произведению трех частей)
decomposition = seasonal_decompose(df.PAY, model='multiplicative')
decomposition.plot()
plt.show()

In [ ]:
# Применяем разложение на тренд, сезонность и остаток для аддитивной модели 
#    (временной ряд = сумме трех частей)
decomposition = seasonal_decompose(df.PAY, model='additive')
decomposition.plot()
plt.show() 

Трендовая и сезонная части выглядят примерно одинаково, но остаточная часть при мультипликативном варианте имеет несколько обособленных выбросов. Аддитивная модель выглядит компактнее и предпочтительнее для раздельного предсказания каждой части.   
Сезонная составляющая имеет слишком малый период и поэтому не позволяет рассмотреть ее на столь малом размере графика.

In [ ]:
# Выделим отдельные составляющие для более детального рассмотрения
trend_part = decomposition.trend       # трендовая составляющаяя
seasonal_part = decomposition.seasonal # сезонная составляющаяя
residual_part = decomposition.resid    # шум

In [ ]:
# Удаляем пустые значения
trend_part = trend_part[~trend_part.isnull()]
seasonal_part = seasonal_part[~seasonal_part.isnull()]
residual_part = residual_part[~residual_part.isnull()]

# Проверяем длины рядов
print()
print('Полный ряд: %d' % len(df))
print('Тренд: %d' % len(trend_part))
print('Сезонность: %d' % len(seasonal_part))
print('Шум: %d' % len(residual_part))

In [ ]:
# Графики составляющих декомпозиции ряда
f, ax = plt.subplots(3, 1, figsize=(20, 20))
ax[0].plot(trend_part, c='g')
ax[0].set_title('Тренд')
ax[1].plot(seasonal_part, c='b')
ax[1].set_title('Сезонность')
ax[2].plot(residual_part, c='r')
ax[2].set_title('Остаток')

Ряд достаточно сложный: и трендовая часть, и остаточная содержат сезонные компоненты, вследствие чего совершают колебания. Это означает, что применение классического метода ARIMA покажет некачественный результат предсказания и лучше применять вариант ARIMA, дополненный учетом сезонных колебаний - SARIMAX.   
     
Ввиду того, что тренд и остаток цикличны, можно попробовать их так же разложить на отдельные компоненты.

In [ ]:
# Применяем разложение на тренд, сезонность и остаток для трендовой части главного ряда
decomposition_trend = seasonal_decompose(trend_part, model='additive')
decomposition_trend.plot()
plt.show()

In [ ]:
# Применяем разложение на тренд, сезонность и остаток для остаточной части главного ряда
decomposition_res = seasonal_decompose(residual_part, model='additive')
decomposition_res.plot()
plt.show()

Долгие эксперименты показали, что в итоге наш главный ряд является фрактальным. Любое сколь угодно глубокое разложение отдельных его составляющих и отдельных частей этих составляющих не убирает сезонные колебания и сохраняет виды колебаний практически в неизменном виде.    
По этой причине вместо классического ML далее упор будет делаться на глубокое обучение.

In [ ]:
# Сезонность колеблется слишком часто. Разглядим ее подробнее на малом отрезке времени
plt.figure(figsize=(20,5))
dfsea = df[(df.index>="2016-07-01") & (df.index<="2016-09-01")]
seasonal=seasonal_decompose(dfsea[['PAY']]).seasonal
seasonal.plot()
plt.show()

In [ ]:
# Посмотрим масштабнее для тренда
plt.figure(figsize=(20,5))
dfsea = df[(df.index>="2016-07-01") & (df.index<="2016-09-01")]
trend=seasonal_decompose(dfsea[['PAY']]).trend
trend.plot()
plt.show()

В сезонности отчетливо просматриваются недельные отрезки с выходными днями. По выходным оплаты проводится мало. 

Проверим стационарность главного ряда и тренда.

In [ ]:
# Тест Дикки-Фуллера
adf = sm.tsa.adfuller(df.PAY) 
p_value = adf[1]
print ('p-value: ', p_value)
if p_value > 0.05: 
    print('Главный ряд не стационарен.')
else:
    print('Главный ряд стационарен.')
    
print()

adf = sm.tsa.adfuller(trend_part) 
p_value = adf[1]
print ('p-value: ', p_value)
if p_value > 0.05: 
    print('Ряд тренда не стационарен.')
else:
    print('Ряд тренда стационарен.')

In [ ]:
# Визуализируем автокорреляцию и частичную автокорреляцию ряда
f, ax = plt.subplots(2, 1, figsize=(20, 20))
sm.graphics.tsa.plot_acf(df.PAY.squeeze(), lags=200, ax=ax[0])
sm.graphics.tsa.plot_pacf(df.PAY.squeeze(), lags=200, ax=ax[1])
plt.show()

Графики автокорреляции показывают явную сезонность и, следовательно, нестационарность ряда. Дифференцирование ряда не улучшило ситуацию. Был проведен эксперимент с дифференцированием 100 раз, но периодичность автокорреляции осталась, и в доверительный интервал 0.05 автокорреляция дифференцированного ряда так и не попала.

In [ ]:
# Эксперимент: дифференцирование ряда 100 раз
i = 1
pay_d = df.PAY
while i<101:
    pay_d = pay_d.diff().iloc[1:]
    i += 1

f, ax = plt.subplots(2, 1, figsize=(20, 10))
sm.graphics.tsa.plot_acf(pay_d.squeeze(), lags=200, ax=ax[0])
sm.graphics.tsa.plot_pacf(pay_d.squeeze(), lags=200, ax=ax[1])
plt.show()
print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(pay_d)[1])

Ряд фрактальный и предсказания для него лучше строить при помощи нейронных сетей.

# Предсказание методом SARIMAX

Для предсказания будем использовать SARIMAX, который умеет учитывать сезонность.   
Параметры p, d, q по графику автокорреляции не определяются, так как не обнаружен "хвост", попадающий в доверительный интервал. Опытным путем были подобраны параметры, которые дают более или менее подходящую точность. Последний параметр "s" установлен = 12, так как глобальный период сезонности = 12 месяцев.     
   
Нормализация данных здесь не использовалась, хотя проверка модели на нормализованных и даже прологарифмированных данных тоже проводилась. Логарифмирование существенно ухудшало метрику, а нормализация вообще на неё не влияла.    
      
Метрика при параметре p = 10 получалась самая низкая из всех опробованных: mape = 40.01

In [ ]:
# Строим модель, обучаем и предсказываем
arima = sm.tsa.statespace.SARIMAX(df.PAY[df.index<START_DATE],
                                order=(0, 0, 1),
                                seasonal_order=(4, 1, 1, 12), # 10 - лучше. Метрика = 40
                                enforce_stationarity=False,
                                enforce_invertibility=False)
arima_result = arima.fit()
arima_pred = arima_result.predict(start = len(df)-PRED_LEN, end = len(df)-1, typ="levels").rename("SARIMAX Predictions")

In [ ]:
# Сравниваем графики тестовых истинных значений и предсказания
plot_prediction(df, arima_pred[arima_pred.index>=START_DATE])

# Проверяем метрику
mape(df[df.index>=START_DATE].PAY, arima_pred[arima_pred.index>=START_DATE])

mape = 51.83

Метрика **mape** выбрана по той причине, что она показывает относительную ошибку. Прочие метрики показывают абсолютные значения, что выдает большие значения из-за больших сумм ряда.

Попробуем получить отдельные предсказания для составлящих декомпозированного ряда и затем сложить их.

In [ ]:
# Предсказываем для тренда
arima_trend = sm.tsa.statespace.SARIMAX(trend_part[trend_part.index<START_DATE],
                                order=(0, 0, 1),
                                seasonal_order=(4, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

result_trend = arima_trend.fit()
arima_pred_trend = result_trend.predict(start = PRED_LEN, end = len(df)-1, typ="levels").rename("PAY")

maxIndex = trend_part.index.max()
plot_prediction(trend_part.to_frame(name='PAY'), 
                arima_pred_trend[(arima_pred_trend.index>=START_DATE) & (arima_pred_trend.index<=maxIndex)])


mape(trend_part[trend_part.index>=START_DATE].values, 
     arima_pred_trend[(arima_pred_trend.index>=START_DATE) & (arima_pred_trend.index<=maxIndex)])

mape для тренда = 15.56

In [ ]:
# Предсказание для сезонности
arima_seasonal = sm.tsa.statespace.SARIMAX(seasonal_part[seasonal_part.index<START_DATE],
                                order=(0, 0, 1),
                                seasonal_order=(6, 1, 1, 12),
                                enforce_stationarity=True,
                                enforce_invertibility=False)

results_seasonal = arima_seasonal.fit()
arima_pred_seasonal = results_seasonal.predict(start = PRED_LEN, end = len(df)-1, typ="levels")

plot_prediction(seasonal_part.to_frame(name='PAY'), arima_pred_seasonal[arima_pred_seasonal.index>=START_DATE])
mape(seasonal_part[seasonal_part.index>=START_DATE].values, arima_pred_seasonal[arima_pred_seasonal.index>=START_DATE])

mape для сезонности = 24.35

In [ ]:
# Предсказание для шума (который тоже, на самом деле, имеет закономерности)
arima_residual = sm.tsa.statespace.SARIMAX(residual_part[residual_part.index<START_DATE],
                                order=(0, 0, 1),
                                seasonal_order=(4, 1, 1, 12),
                                enforce_stationarity=True,
                                enforce_invertibility=False)

results_residual = arima_residual.fit()
arima_pred_residual = results_residual.predict(start = PRED_LEN, end = len(df)-1, typ="levels")

maxIndex = residual_part.index.max()
plot_prediction(residual_part.to_frame(name='PAY'), 
                arima_pred_residual[(arima_pred_residual.index>=START_DATE) & (arima_pred_residual.index<=maxIndex)])
mape(residual_part[residual_part.index>=START_DATE].values, 
     arima_pred_residual[(arima_pred_residual.index>=START_DATE) & (arima_pred_residual.index<=maxIndex)])

Шум спрогнозировать не удалось, но на то он и шум.   
mape для остатка = 105.33

In [ ]:
# Составим итоговое предсказание как сумму трех предсказаний
pred_full = arima_pred_trend + arima_pred_seasonal + arima_pred_residual
pred_full = pred_full.dropna()

plot_prediction(df, pred_full[pred_full.index>=START_DATE])

mape(df[(df.index>=START_DATE) & (df.index<=maxIndex)].PAY, 
     pred_full[(pred_full.index>=START_DATE) & (pred_full.index<=maxIndex)])

Раздельное предсказание каждой части и последующее суммирование показало себя лучше, чем предсказание неразделенного ряда.    
mape = 27.03

In [ ]:
# Попробуем предсказать без учета шума
pred_full = arima_pred_trend + arima_pred_seasonal # + arima_pred_residual
pred_full = pred_full.dropna()

plot_prediction(df, pred_full[pred_full.index>=START_DATE])

mape(df[(df.index>=START_DATE) & (df.index<=maxIndex)].PAY, 
     pred_full[(pred_full.index>=START_DATE) & (pred_full.index<=maxIndex)])

Исключение шума из прогноза практически не изменило оценку предсказания.    
mape = 26.97

In [ ]:
# Сохраним модели для итогового предсказания в ансамбле
# result_trend.save("result_trend.pickle")
# results_seasonal.save("results_seasonal.pickle")

# Предсказание с использованием DL

### Нормализация и разделение данных

Перед тем, как применять модели глубокого обучения, нормализуем данные, поскольку DL чувствительно к большим разбросам в значениях.

In [ ]:
# Нормализуем
scaler = MinMaxScaler()
df_scal = scaler.fit_transform(df)
df_scal = pd.DataFrame(df_scal, columns=df.columns)

# Разделим на тренировочную и тестовую части
test = df_scal[df.index>=START_DATE]
train = df_scal[df.index<START_DATE]

Для построения нейронных сетей особым образом организуем данные. Каждому числу во временном ряду сопоставим набор признаков - предыдущие 30 дней. Будем считать, что именно за месяц происходят основные изменения в поведении данных. Для более глубокого обучения можно взять другой размер окна, например, = 365, то есть, год. Но это значительно увеличивает вычислительные ресурсы.

In [ ]:
# Генерируем набор признаков из предыдущих значений для обучающей выборки
train_x = []
for i in range(INPUT_LEN, len(train)):
    train_x.append(train.iloc[i-INPUT_LEN:i])

train_y = train.iloc[INPUT_LEN:]

train_x = np.array(train_x)
train_y = np.array(train_y)

train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))

In [ ]:
# Генерируем набор признаков из предыдущих значений для тестовой выборки
test_x = []
for i in range(INPUT_LEN, len(test)):
    test_x.append(test.iloc[i-INPUT_LEN:i])

test_y = test.iloc[INPUT_LEN:]

test_x = np.array(test_x)
test_y = np.array(test_y)

test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))

In [ ]:
# Проверим размеры выборок
print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)

### CNN

Попробуем построить предсказание, используя свёрточные нейронные сети. В этих сетях предполагается, что все входы и выходы независимы. Однако в нашем случае входы - это скользящее окно, поэтому память о зависимостях между элементами ряда всё равно присутствует.

In [ ]:
# Свёрточная сеть №1 
model_conv1 = Sequential()
model_conv1.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(INPUT_LEN, 1)))
model_conv1.add(MaxPooling1D(pool_size=2))
model_conv1.add(Flatten())
model_conv1.add(Dense(128, activation='relu'))
model_conv1.add(Dense(64, activation='relu'))
model_conv1.add(Dense(1))
model_conv1.compile(optimizer='adam', loss='mape')

In [ ]:
# Callback
checkpoint_conv1 = ModelCheckpoint('../working/conv1.hdf5' , monitor=['mape'], verbose=0  , mode='min')
earlystop = EarlyStopping(monitor='mape', mode = "min", patience=10)
   
callbacks_list = [checkpoint_conv1, earlystop]

In [ ]:
# Обучаем
model_conv1.fit(train_x, train_y, epochs=200, verbose=1, callbacks=callbacks_list)

In [ ]:
# Делаем предсказание и проверяем его
yhat = model_conv1.predict(test_x, verbose=0)

temp = df_scal[-PRED_LEN+INPUT_LEN:]
temp.PAY = yhat
prediction_conv1 = pd.DataFrame(scaler.inverse_transform(temp), columns=df_scal.columns)

plot_prediction(df[-PRED_LEN+INPUT_LEN:], prediction_conv1.PAY)
mape(df.PAY[-PRED_LEN+INPUT_LEN:],prediction_conv1.PAY)

mape лежит в пределах от 22.19 до 24.50

In [ ]:
# Свёрточная сеть №2. Добавим еще больше свёрточных слоёв
model_conv2 = Sequential()
model_conv2.add(Conv1D(filters=128, kernel_size=2, activation='relu', input_shape=(INPUT_LEN, 1)))
model_conv2.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
model_conv2.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
model_conv2.add(MaxPooling1D(pool_size=2))
model_conv2.add(Flatten())
model_conv2.add(Dense(100, activation='relu'))
model_conv2.add(Dense(1))
model_conv2.compile(optimizer='adam', loss='mape')

In [ ]:
# Callback и обучение
checkpoint_conv2 = ModelCheckpoint('../working/conv2.hdf5' , monitor=['mape'], verbose=0  , mode='min')
callbacks_list = [checkpoint_conv2, earlystop]

# fit model
model_conv2.fit(train_x, train_y, epochs=200, verbose=1, callbacks=callbacks_list)

In [ ]:
# Делаем предсказание и проверяем его
yhat = model_conv2.predict(test_x, verbose=0)

temp = df_scal[-PRED_LEN+INPUT_LEN:]
temp.PAY = yhat
prediction_conv2 = pd.DataFrame(scaler.inverse_transform(temp), columns=df_scal.columns)

plot_prediction(df[-PRED_LEN+INPUT_LEN:], prediction_conv2.PAY)
mape(df.PAY[-PRED_LEN+INPUT_LEN:],prediction_conv2.PAY)

Эта модель показывает примерно такой же результат: mape = от 22 до 24

### RNN

Рекуррентные сети оперируют последовательностями входов и эти последовательности обрабатываются целиком.     
Сначала построим простую рекуррентную сеть, а затем проверим, как предсказывает сеть с долговременной памятью LSTM.

In [ ]:
# Простая рекуррентная сеть
model_rnn = Sequential()
model_rnn.add(SimpleRNN(48,activation="tanh",return_sequences=True, input_shape=(train_x.shape[1],1)))
model_rnn.add(Dropout(0.2))
model_rnn.add(SimpleRNN(48,activation="tanh",return_sequences=True))
model_rnn.add(Dropout(0.02))
model_rnn.add(SimpleRNN(48,activation="tanh",return_sequences=False))
model_rnn.add(Dropout(0.02))
model_rnn.add(Dense(1))

optimizer = optimizers.RMSprop(lr=0.001, clipvalue=1.0)
model_rnn.compile(loss = "mse", optimizer = optimizer, metrics =['mae', 'mse', 'mape'])
# model_rnn.compile(optimizer="adam",loss="mape")

In [ ]:
# Callback и обучение
checkpoint_rnn = ModelCheckpoint('../working/rnn.hdf5' , monitor=['mape'], verbose=0  , mode='min')
callbacks_list = [checkpoint_rnn, earlystop]

model_rnn.fit(train_x, train_y, epochs=20, batch_size=1, callbacks=callbacks_list)

In [ ]:
# Предсказываем и проверяем
yhat = model_rnn.predict(test_x, batch_size=1)

temp = df_scal[-PRED_LEN+INPUT_LEN:]
temp.PAY = yhat
prediction_rnn = pd.DataFrame(scaler.inverse_transform(temp), columns=df_scal.columns)

plot_prediction(df[-PRED_LEN+INPUT_LEN:], prediction_rnn.PAY)
mape(df.PAY[-PRED_LEN+INPUT_LEN:],prediction_rnn.PAY)

Метрика mape примерно такая же, как у свёрточных сетей.

Построим простую сеть LSTM.

In [ ]:
# LSTM-модель №1
model_lstm = Sequential()
model_lstm.add(LSTM(units=10, stateful=True, return_sequences=True, batch_input_shape=(1, train_x.shape[1], 1)))
model_lstm.add(LSTM(units=10, stateful=True, return_sequences=True))
model_lstm.add(LSTM(units=10, stateful=True, return_sequences=True))
model_lstm.add(LSTM(units=10))
model_lstm.add(Dense(units=1))

# model_lstm.compile(optimizer="adam", loss="mape") # RMSprop
optimizer = optimizers.RMSprop(lr=0.001, clipvalue=1.0)
model_lstm.compile(loss = "mse", optimizer = optimizer, metrics =['mape'])

In [ ]:
# Callback и обучение
checkpoint_lstm = ModelCheckpoint('../working/lstm.hdf5' , monitor=['mape'], verbose=0  , mode='min')
callbacks_list = [checkpoint_lstm, earlystop]

model_lstm.fit(train_x, train_y, epochs=5, batch_size=1, callbacks=callbacks_list)

In [ ]:
# Предсказываем и проверяем
yhat = model_lstm.predict(test_x, batch_size=1)

temp = df_scal[-PRED_LEN+INPUT_LEN:]
temp.PAY = yhat
prediction_lstm = pd.DataFrame(scaler.inverse_transform(temp), columns=df_scal.columns)

plot_prediction(df[-PRED_LEN+INPUT_LEN:], prediction_lstm.PAY)
mape(df.PAY[-PRED_LEN+INPUT_LEN:],prediction_lstm.PAY)

По монитору обучения,а также по графику видно, что сеть не очень успешно обучается, и метрика на выходе не очень хорошая.         
       
Попробуем применить двунаправленную сеть LSTM.

In [ ]:
# LSTM-модель №2
model_bi_lstm = Sequential()
model_bi_lstm.add(Bidirectional(LSTM(units=48, stateful=True, return_sequences=True, 
                          input_shape=(train_x.shape[1],1))))
model_bi_lstm.add(Dropout(0.2))
model_bi_lstm.add(Bidirectional(LSTM(units=48, stateful=True, return_sequences=True)))
model_bi_lstm.add(Dropout(0.2))
model_bi_lstm.add(Bidirectional(LSTM(units=48, stateful=True, return_sequences=True)))
model_bi_lstm.add(Dropout(0.2))
model_bi_lstm.add(Bidirectional(LSTM(units=48)))
model_bi_lstm.add(Dropout(0.2))
model_bi_lstm.add(Dense(units=1))

# 10 = 38, 32 = 32 # 32 = 27.31146940644043, 48 = 26.569509165594816
optimizer = optimizers.RMSprop(lr=0.001, clipvalue=1.0)
# model_bi_lstm.compile(optimizer="adam", loss="mape") # RMSprop
model_bi_lstm.compile(loss = "mse", optimizer = optimizer, metrics =['mae', 'mse', 'mape'])

In [ ]:
# Callback и обучение
checkpoint_bi_lstm = ModelCheckpoint('../working/bi_lstm.hdf5' , monitor=['mape'], verbose=0  , mode='min')
callbacks_list = [checkpoint_bi_lstm, earlystop]

model_bi_lstm.fit(train_x, train_y, epochs=5, batch_size=1, callbacks=callbacks_list)

In [ ]:
# Предсказываем и проверяем
yhat = model_bi_lstm.predict(test_x, batch_size=1)

temp = df_scal[-PRED_LEN+INPUT_LEN:]
temp.PAY = yhat
prediction_bi_lstm = pd.DataFrame(scaler.inverse_transform(temp), columns=df_scal.columns)

plot_prediction(df[-PRED_LEN+INPUT_LEN:], prediction_bi_lstm.PAY)
mape(df.PAY[-PRED_LEN+INPUT_LEN:],prediction_bi_lstm.PAY)

Данная модель работает нестабильно. Иногда метрика предсказания получается неплохая - около 20%, но иногда она выходит почти 100%.

### CNN + RNN

Идея одновременного применения свёртки и рекурретности заключается в том, чтобы понизить размерность, выделив главные связи между входами, а затем уже применять рекурретное обучение.

In [ ]:
# Модель CNN+RNN №1
model_sr1 = Sequential()
model_sr1.add(Conv1D(filters=256, kernel_size=2, activation='relu', input_shape=(30,1)))
model_sr1.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
model_sr1.add(MaxPooling1D(pool_size=2))
model_sr1.add(Flatten())
model_sr1.add(RepeatVector(30))
model_sr1.add(LSTM(128, activation='relu'))
model_sr1.add(Dense(100, activation='relu'))
model_sr1.add(Dense(1))
# model_sr1.compile(loss='mape', optimizer='adam')

model_sr1.compile(loss = "mse", optimizer = optimizer, metrics =['mae', 'mse', 'mape'])

In [ ]:
# Callback и обучение
checkpoint_sr1 = ModelCheckpoint('../working/sr1.hdf5' , monitor=['mape'], verbose=0  , mode='min')
callbacks_list = [checkpoint_sr1, earlystop]

model_sr1.fit(train_x, train_y, epochs=10, callbacks=callbacks_list)

In [ ]:
# Предсказываем и проверяем
yhat = model_sr1.predict(test_x, batch_size=1)

temp = df_scal[-PRED_LEN+INPUT_LEN:]
temp.PAY = yhat
prediction_sr1 = pd.DataFrame(scaler.inverse_transform(temp), columns=df_scal.columns)

plot_prediction(df[-PRED_LEN+INPUT_LEN:], prediction_sr1.PAY)
mape(df.PAY[-PRED_LEN+INPUT_LEN:],prediction_sr1.PAY)

mape = 19.86

In [ ]:
# Модель CNN+RNN №2
model_sr2 = Sequential()
model_sr2.add(Conv1D(filters=256, kernel_size=2, activation='relu', input_shape=(30,1)))
model_sr2.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
model_sr2.add(MaxPooling1D(pool_size=2))
model_sr2.add(Flatten())
model_sr2.add(RepeatVector(30))
model_sr2.add(LSTM(units=100, return_sequences=True, activation='relu'))
model_sr2.add(Dropout(0.2))
model_sr2.add(LSTM(units=100, return_sequences=True, activation='relu'))
model_sr2.add(Dropout(0.2))
model_sr2.add(LSTM(units=100, return_sequences=True, activation='relu'))
model_sr2.add(LSTM(units=100, return_sequences=True, activation='relu'))
model_sr2.add(Bidirectional(LSTM(128, activation='relu')))
model_sr2.add(Dense(100, activation='relu'))
model_sr2.add(Dense(1))
model_sr2.compile(loss = "mse", optimizer = optimizer, metrics =['mae', 'mse', 'mape'])

In [ ]:
# Callback и обучение
checkpoint_sr2 = ModelCheckpoint('../working/sr2.hdf5' , monitor=['mape'], verbose=0  , mode='min')
callbacks_list = [checkpoint_sr2, earlystop]

model_sr2.fit(train_x, train_y, epochs=15, callbacks=callbacks_list)

In [ ]:
# Предсказываем и проверяем
yhat = model_sr2.predict(test_x, batch_size=1)

temp = df_scal[-PRED_LEN+INPUT_LEN:]
temp.PAY = yhat
prediction_sr2 = pd.DataFrame(scaler.inverse_transform(temp), columns=df_scal.columns)

plot_prediction(df[-PRED_LEN+INPUT_LEN:], prediction_sr2.PAY)
mape(df.PAY[-PRED_LEN+INPUT_LEN:],prediction_sr2.PAY)

mape = 22.64

**Выводы по использованию моделей ML и DL:**     
1. Чистая модель SARIMAX предсказывает не очень хорошо. Её оценка mape = 40-50%. Но декомпозиция ряда и прогнозирование отдельных составляющих с последующим сложением уменьшает оценку до 27%.     
2. Модели DL существенно лучше дают прогноз, нежели ML-модель SARIMAX. Оценка Возможно, и SARIMAX даст неплохие предсказания, если увеличить параметр p. При его увеличении SARIMAX соответственно улучшает и оценку. Однако при p=10 вычисление затягивается почти на час (на платформе Kaggle), а при p=12 возникает ошибка переполнения памяти.     
3. Чистая модель RNN и однонаправленная модель LSTM с предсказанием справляются плохо. Чистые свёрточные модели предсказывают стабильно на уровне 23-26%.     
4. Сети с архитектурой CNN+RNN дают наилучший прогноз. Их оценка 19-20%.

# Предсказание с использованием специальной модели Prophet

Модель Prophet разработана командой Facebook специально для обработки временных рядов. Она построена не на нейронных сетях, а на ML-подходах. Главная особенность Prophet - это умение модели раскладывать временной ряд на сумму рядов: тренд, сезонность, аномальные дни и остаток. Известно, что годовая сезонность моделируется в Prophet рядами Фурье, а сезонность - кусочно-линейными функциями.     
     
Prophet позволяет учитывать выходные, праздники и дни, когда в ряду показатели либо экстремально падают, либо вырастают. Кроме того, Prophet умеет обрабатывать дополнительные регрессоры, то есть признаки, которые сопутствуют главному показателю (в нашем случае это оплаты) и влияют на его поведение.     
     
Данные для Prophet можно не нормализовать, так как это делает сама модель. Это было проверено опытным путём. Результаты на исходных данных и нормализованных полностью совпадали.

Для работы с Prophet временной ряд должен иметь стандартные названия для двух признаков: **ds** - для даты и **y** - для прогнозируемого признака.

In [ ]:
# Готовим данные для Prophet - переименовываем признаки
df_pr = df.copy()
df_pr = df_pr.reset_index()
df_pr.columns = ['ds','y'] 

# Разбиваем на тренировочные и тестовые
train_data_pr = df_pr.iloc[:len(df)-PRED_LEN]
test_data_pr = df_pr.iloc[len(df)-PRED_LEN:]

Для учёта нерабочих дней загрузим из библиотеки holidays данные о праздничных днях в России. Эта библиотека не учитывает переносы выходных на другие дни, принятые у нас в стране, но лучше иметь неполную информацию, чем не иметь её вовсе.    
Для применения списка выходных дней в модели, их тоже необходимо назвать специальным образом.

In [ ]:
# Датасет для передачи параметром в модель
holiday = pd.DataFrame()

# Получаем данные о выходных днях и готовим для модели
for date, name in sorted(holidays.Russia(years=list(range(2008,2022))).items()):
    holiday = holiday.append(pd.DataFrame({'ds': date, 'holiday': "Ru-Holidays"}, index=[0]), ignore_index=True)
holiday['ds'] = pd.to_datetime(holiday['ds'], format='%Y-%m-%d', errors='ignore')

holiday

Для учета аномальных дней можно построить дополнительные признаки и указать модели, что их надо учитывать при прогнозировании.     
      
Аномальными будем считать воскресенья, когда кассы ЭСК закрыты и оплата поступает только через онлайн-сервисы. Также укажем, что до 12-го числа оплат очень мало (это связано с тем, что счета выставляют в начале месяца и их не спешат сразу оплачивать, ожидая зарплаты). Дополнительно укажем, что в промежутке между 17 и 25 числом оплат, напротив, очень много.   
    
Кроме того, укажем модели, что надо при прогнозировании учитывать день недели и день в году. То, что эти признаки важны, получено опытным путём. Ниже на графиках показаны сезонные колебания прогнозируемого признака, на основании которых строились дополнительные регрессоры.

In [ ]:
# Временный набор данных для исследования сезонности
dft = df.copy().reset_index()
dft['year'] = dft['Date'].dt.year
dft['month'] = dft['Date'].dt.month
dft['day'] = dft['Date'].dt.day
dft['day_of_year'] = dft['Date'].dt.dayofyear
dft['week_of_year'] = dft['Date'].dt.weekofyear
dft['quarter'] = dft['Date'].dt.quarter
dft['season'] = dft.month%12 // 3 + 1
dft['day_of_week'] = dft['Date'].dt.dayofweek

dft[['Date', 'year', 'month', 'day', 'day_of_year', 'week_of_year', 'quarter', 'season', 'day_of_week']].head()

In [ ]:
# Графики для проверки сезонности по новым признакам
for col in ['year', 'month', 'day', 'day_of_year', 'week_of_year', 'quarter', 'season', 'day_of_week']:
    fig, ax = plt.subplots(1, 2, figsize=(7,3))
    ax[0].set_title('Mean ' + col)
    dft.groupby(by = col).mean()['PAY'].plot(ax=ax[0], kind='bar')
    ax[1].set_title('Median ' + col)
    dft.groupby(by = col).median()['PAY'].plot(ax=ax[1], kind='bar')

Тренд по годам - почти линейный. Причина - инфляция и рост тарифов.     
По месяцам - оплаты падают летом и резко возрастают в декабре перед окончанием года и в феврале, когда приходят первые деньги после новогодних каникул.     
Графики по дням в году, по неделям в году, по сезонам и по кварталам практически совпадают с графиком месяцев в году. Поэтому все эти графики избыточны.   
На графики распределения величины по дням месяца и дням недели посмотрим подробнее, построив их в другом масштабе.

In [ ]:
# Графики более подробного рассмотрения сезонности по дням месяца и дням недели
fig, ax = plt.subplots(1, 2, figsize=(16,8))
ax[0].set_title('Mean day' )
dft.groupby(by = 'day').mean()['PAY'].plot(ax=ax[0], kind='bar')
ax[1].set_title('Mean day of week' )
dft.groupby(by = 'day_of_week').median()['PAY'].plot(ax=ax[1], kind='bar')

График оплат по дням месяца показывает, что оплат значительно меньше до 12-го числа включительно и существенно больше в интервале с 17 по 25 число.    
График оплаты по дням недели резко выделяет только один день - воскресенье.     
     
Учтём все эти наблюдения при построении регрессоров для Prophet.

In [ ]:
# Функция для построения признака воскресенья
def sunday_date(ds):
    date = pd.to_datetime(ds)
    if date.weekday() == 6:
        return 1
    else:
        return 0

# Функция для построения признака дней, когда поступления оплат резко падают
def bad_date(ds):
    date = pd.to_datetime(ds)
    if date.day < 12:
        return 1
    else:
        return 0

# Функция для построения признака дней, когда поступления оплат резко возрастают
def good_date(ds):
    date = pd.to_datetime(ds)
    if date.day <= 25 and date.day >=17:
        return 1
    else:
        return 0

In [ ]:
# Добавляем дополнительные признаки, влияющие на прогноз
train_data_pr['day_of_year'] = train_data_pr['ds'].dt.dayofyear
train_data_pr['day_of_week'] = train_data_pr['ds'].dt.day_of_week
train_data_pr['sunday_date'] = train_data_pr['ds'].apply(sunday_date)
train_data_pr['bad_date'] = train_data_pr['ds'].apply(bad_date)
train_data_pr['good_date'] = train_data_pr['ds'].apply(good_date)

Оптимальные параметры для Prophet были получены при помощи самодельного GridSearch. Код поиска параметров приводится ниже в виде закомментаренных ячеек.

In [ ]:
# Модель Prophet
model_prop = Prophet(holidays=holiday,        # учёт праздников 
             changepoint_prior_scale = 0.001, # шаг сетки для поисков локальных экстремумов
             weekly_seasonality = True,       # учёт недельной сезонности
             yearly_seasonality = True,       # учет годовой сезонности
             n_changepoints = 100             # количество экстремумов (не очень ясный параметр)
            )

# Добавляем регрессоры, то есть указываем, какие дополнительные признаки влияют на прогноз
model_prop.add_regressor('sunday_date')
model_prop.add_regressor('bad_date')
model_prop.add_regressor('good_date')
model_prop.add_regressor('day_of_year')
model_prop.add_regressor('day_of_week')

In [ ]:
# Обучаем модель
model_prop.fit(train_data_pr)

# Строим временной ряд, который содержит тренировочные даты + предсказываемые даты
future = model_prop.make_future_dataframe(periods=PRED_LEN)
future

In [ ]:
# К ряду также добавляем дополнительные признаки
future['sunday_date'] = future['ds'].apply(sunday_date)
future['bad_date'] = future['ds'].apply(bad_date)
future['good_date'] = future['ds'].apply(good_date)
future['day_of_year'] = future['ds'].dt.dayofyear
future['day_of_week'] = future['ds'].dt.day_of_week

In [ ]:
# Предсказываем и проверяем результат
forecast = model_prop.predict(future) 
forecast

In [ ]:
# Визуализируем прогноз. Синим показано предсказание, чёрным - исходные данные
fig1 = model_prop.plot(forecast)

In [ ]:
# Визуализируем составлющие ряда, выделенные моделью 
fig2 = model_prop.plot_components(forecast)

Выданный прогноз содержит множество информации. Нас интересует столбец yhat, так как в нем находится прогноз для текущей даты.

In [ ]:
# Выделяем из полного прогноза ряд предсказания
prediction_prop = forecast[forecast.ds>=START_DATE][['ds','yhat']]
prediction_prop = prediction_prop.rename(columns = {'ds': 'Date', 'yhat': 'PAY'})
prediction_prop = prediction_prop.set_index('Date')

# График и оценка
plot_prediction(df, prediction_prop.PAY)
mape(df[df.index>=START_DATE].PAY, prediction_prop.PAY) 

По графику видно, что Prophet не справляется с пиковыми значениями величины. Это известная проблема - Prophet считает эти значения выбросами и игнорирует их. Решение проблемы пока не найдено. Возможно, надо более тщательно подобрать параметры.

In [ ]:
# Сохраним модель
# with open('serialized_model.json', 'w') as fout:
#     json.dump(model_to_json(m), fout)  

# К сожалению, при сохранении происходит ошибка, которую непонятно как исправлять

# Воспользуемся простым pickle
# with open('prophet.pickle', 'wb') as f:
#     pickle.dump(model_prop, f)

Для уменьшения значений в пиковых точках попробуем прологарифмировать наш ряд.

In [ ]:
# Повторяем все действия на прологарифмированном ряде
train_data_pr_log = train_data_pr.copy()
train_data_pr_log.y = np.log(train_data_pr.y)

model_prop_log = Prophet(holidays=holiday,     
             changepoint_prior_scale = 0.001, 
             weekly_seasonality = True,       
             yearly_seasonality = True,       
             n_changepoints = 100             
            )

# Добавляем регрессоры, то есть указываем, какие дополнительные признаки влияют на прогноз
model_prop_log.add_regressor('sunday_date')
model_prop_log.add_regressor('bad_date')
model_prop_log.add_regressor('good_date')
model_prop_log.add_regressor('day_of_year')
model_prop_log.add_regressor('day_of_week')

# Обучаем модель
model_prop_log.fit(train_data_pr_log)

# Предсказываем и проверяем результат
forecast = model_prop_log.predict(future) 

# Выделяем из полного прогноза ряд предсказания
prediction_prop_log = forecast[forecast.ds>=START_DATE][['ds','yhat']]
prediction_prop_log = prediction_prop_log.rename(columns = {'ds': 'Date', 'yhat': 'PAY'})
prediction_prop_log = prediction_prop_log.set_index('Date')
prediction_prop_log.PAY = np.exp(prediction_prop_log.PAY)

# График и оценка
plot_prediction(df, prediction_prop_log)
mape(df[df.index>=START_DATE].PAY, prediction_prop_log.PAY)

Метрика ухудшилась, хотя к верхним пикам предсказания подтянулись. Но также предсказания излишне вытянулись в нижних значениях. В целом лограифмирование не помогло.

Ниже приводится код, при помощи которого выполнялся поиск оптимальных параметров для модели Prophet.

In [ ]:
# from sklearn.model_selection import ParameterGrid
# params_grid = {'seasonality_mode':('multiplicative','additive'),
#                'changepoint_prior_scale':[0.01,0.05,0.1],
#                'n_changepoints' : [50,100],
#                'weekly_seasonality' : [True,False],
#                'daily_seasonality' : [True,False],
#                'yearly_seasonality' : [True,False],
#               }
# grid = ParameterGrid(params_grid)
# cnt = 0
# for p in grid:
#     cnt = cnt+1

# print('Total Possible Models',cnt)

Total Possible Models 96

In [ ]:
# model_parameters = pd.DataFrame(columns = ['MAPE','Parameters'])
# for p in grid:
#     print(p)
#     np.random.seed(0)
#     m7 = Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
#                          n_changepoints = p['n_changepoints'],
#                          seasonality_mode = 'additive',
#                          weekly_seasonality= p['weekly_seasonality'],
#                          daily_seasonality = p['daily_seasonality'],
#                          yearly_seasonality = p['yearly_seasonality'],
#                          holidays=holiday, 
#                          interval_width=0.95)
    
#     m7.add_regressor('nfl_sunday')
#     m7.add_regressor('nfl_bad_date')
#     m7.add_regressor('nfl_good_date')
#     m7.fit(train_data_pr)
#     future7 = m7.make_future_dataframe(periods=PRED_LEN)
#     future7['nfl_sunday'] = future7['ds'].apply(nfl_sunday)
#     future7['nfl_bad_date'] = future7['ds'].apply(nfl_bad_date)
#     future7['nfl_good_date'] = future7['ds'].apply(nfl_good_date)
#     prophet_pred7 = m7.predict(future7)

#     temp7 = df3[df.index>=START_DATE]
#     temp7.y = prophet_pred7.yhat
#     temp7 = temp7.rename(columns = {'ds': 'Date', 'y': 'PAY'})
#     temp7 = temp7.set_index('Date')
    
#     MAPE = mape(df.loc[df.index>=START_DATE, "PAY"].values, temp7.PAY)
#     print('MAPE------------------------------------',MAPE)
#     model_parameters = model_parameters.append({'MAPE':MAPE,'Parameters':p},ignore_index=True)

Лучший вариант:    
{'changepoint_prior_scale': 0.01, 'daily_seasonality': True, 'n_changepoints': 100, 'seasonality_mode': 'multiplicative', 'weekly_seasonality': True, 'yearly_seasonality': True}
MAPE------------------------------------ 24.862912001211704

По графику видно, что исходная модель (без логарифмирования) немного не догятивает в прогнозе до пиковых экстремумов. Поправим это вручную. Умножим прогноз на небольшой коэффициент для коррекции прогноза в области локальных экстремумов, которые Prophet считает выбросами.

In [ ]:
# Корректируем прогноз в экстремумах
prediction_prop_corr = prediction_prop.copy()
prediction_prop_corr['PAY'] = prediction_prop.apply(lambda row: \
                row['PAY']*1.2 if row.name.day>=17 and row.name.day<=25 else \
                row['PAY']/1.2 if row.name.day<=10 else \
                row['PAY'] , axis=1)

# График и оценка
plot_prediction(df, prediction_prop_corr)
mape(df[df.index>=START_DATE].PAY, prediction_prop_corr.PAY)

Скорректированный прогноз улучшил метрику.

In [ ]:
# prediction61['mPAY'] = prediction61.apply(lambda row: \
# #                 row['PAY']*1.1 if row['dt'].day>=17 and row['dt'].day<=25 else \
# #                 row['PAY']/1.2 if row['dt'].day<=10 else \
#                 row['PAY']*1.03+4000000 , axis=1)

# Ансамбль моделей

Теперь, после того как получили прогнозы при помощи моделей нескольких видов, соединим их в ансамбле. Сначала возьмём простое среднее по всем моделям, а затем из всех предсказаний составим регрессию и построим взвешенное среднее с учётом коэффициентов регрессии.

### Подготовка данных для ансамбля

Для быстроты работы загрузим модели, которые рассчитываются долго, и чей код обучения был взят в комментарии - это нейронные сети.  

In [ ]:
# SARIMAX
# result_trend = SARIMAResults.load('result_trend.pickle')
# result_seasonal = SARIMAResults.load('result_seasonal.pickle')

# arima_pred_trend = result_trend.predict(start = PRED_LEN, end = len(df)-1, typ="levels")
# arima_pred_seasonal = results_seasonal.predict(start = PRED_LEN, end = len(df)-1, typ="levels")

# arima_full = arima_pred_trend + arima_pred_seasonal 
# arima_full = arima_full.dropna()

prediction_sarimax = pred_full
print(prediction_sarimax.head(2))
print(prediction_sarimax.tail(6))

In [ ]:
# Prophet
# with open('serialized_model.json', 'r') as fin:
#     model_prop = model_from_json(json.load(fin))

# with open(pkl_path, 'rb') as f:
#     model_prop = pickle.load(f)
    
print(prediction_prop.head(2))
print(prediction_prop.tail(6))

In [ ]:
# Загрузка моделей NN
model_conv1 = load_model('../input/payments-model/conv1.hdf5')
model_conv2 = load_model('../input/payments-model/conv2.hdf5')
model_rnn = load_model('../input/payments-model/rnn.hdf5')
model_lstm = load_model('../input/payments-model/lstm.hdf5')
model_bi_lstm = load_model('../input/payments-model/bi_lstm.hdf5')
model_sr1 = load_model('../input/payments-model/sr1.hdf5')
model_sr2 = load_model('../input/payments-model/sr2.hdf5')

# Списки моделей и предсказаний
modelsNN = [model_conv1,model_conv2,model_rnn,model_lstm, model_bi_lstm,model_sr1,model_sr2] #
predictionsNN = []

# Предсказания
for i in range(len(modelsNN)):
    m = modelsNN[i]
    if i==3 or i==4:
        yhat = m.predict(test_x, batch_size=1)
    else:
        yhat = m.predict(test_x)
    temp = df_scal[-PRED_LEN+INPUT_LEN:]
    temp.PAY = yhat
    predictionsNN.append(pd.DataFrame(scaler.inverse_transform(temp), columns=df_scal.columns))

### Простое среднее предсказаний

In [ ]:
# Датасет для aнсамбля
prediction = predictionsNN[0].copy()

# Суммируем предсказания нейронных сетей
for i in range(1,len(predictionsNN)):
    prediction = prediction + predictionsNN[i]
    
# Добавляем предсказания SARIMAX и Prophet
# prediction.PAY = prediction.PAY + prediction_sarimax[-PRED_LEN+INPUT_LEN:].reset_index().iloc[:,1].rename('PAY')
prediction.PAY = prediction.PAY + prediction_prop[-PRED_LEN+INPUT_LEN:].reset_index().iloc[:,1]

In [ ]:
# Вычисляем среднее, строим график и проверяем оценку
prediction.PAY = prediction.PAY / (len(predictionsNN)+1)

plot_prediction(df[-PRED_LEN+INPUT_LEN:], prediction)
mape(df[-PRED_LEN+INPUT_LEN:].PAY, prediction.PAY)

mape:    
* для полного ансамбля всех моделей = 17.52
* для ансамбля только из нейронных сетей = 17.36    
* для ансамбля всех моделей без SARIMAX = 16.88

### Регрессия предсказаний

In [ ]:
# Составим обучающие мета-данные для линейной регрессии
meta_train = np.hstack([
    np.array(predictionsNN[0].PAY).reshape(-1,1), 
    np.array(predictionsNN[1].PAY).reshape(-1,1),
    np.array(predictionsNN[2].PAY).reshape(-1,1),
    np.array(predictionsNN[3].PAY).reshape(-1,1),
    np.array(predictionsNN[4].PAY).reshape(-1,1),
    np.array(predictionsNN[5].PAY).reshape(-1,1),
    np.array(predictionsNN[6].PAY).reshape(-1,1),
#     np.array(prediction_sarimax[-PRED_LEN+INPUT_LEN:]).reshape(-1,1),
    np.array(prediction_prop[-PRED_LEN+INPUT_LEN:].PAY).reshape(-1,1)
])

y_meta_train = df[-PRED_LEN+INPUT_LEN:].PAY.values.reshape(-1,1)

In [ ]:
# Мета-модель CatBoost
from catboost import CatBoostRegressor
meta_CatBt = CatBoostRegressor(iterations = 5000,
                          random_seed = 21,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
meta_CatBt.fit(meta_train, y_meta_train, verbose_eval=0)
coeff = meta_CatBt.feature_importances_
coeff = coeff/100
coeff

Коэффициенты:    
[0.11628278, 0.08466846, 0.12683852, 0.07743247, 0.13885572, 0.10315013, 0.13071674, 0.22205518]

In [ ]:
prediction['mean_PAY'] = (
    coeff[0]*predictionsNN[0].PAY + \
    coeff[1]*predictionsNN[1].PAY + \
    coeff[2]*predictionsNN[2].PAY + \
    coeff[3]*predictionsNN[3].PAY + \
    coeff[4]*predictionsNN[4].PAY + \
    coeff[5]*predictionsNN[5].PAY + \
    coeff[6]*predictionsNN[6].PAY + \
#     coeff[7]*prediction_sarimax[-PRED_LEN+INPUT_LEN:].reset_index().iloc[:,1].rename('PAY') + \
    coeff[7]*prediction_prop[-PRED_LEN+INPUT_LEN:].reset_index().iloc[:,1]
)

plot_prediction(df[-PRED_LEN+INPUT_LEN:], prediction['mean_PAY'])
mape(df[-PRED_LEN+INPUT_LEN:].PAY, prediction['mean_PAY'])

mape:    
* для полного ансамбля всех моделей = 16.97
* для ансамбля только из нейронных сетей = 16.91    
* для ансамбля всех моделей без SARIMAX = 16.66

# Предсказание на заданное количество дней

После того, как исследован ряд и подобраны модели, сделаем предсказание на будущее. Предсказание будет выполняться на N дней вперед от конца набора данных, который у нас имеется.     
Заново обучим модели на полном наборе данных и предскажем будущее. С учетом того, что модель SARIMAX даёт не слишком хорошие оценки и без неё метрика лучше, будем использовать ансамбль из моделей NN и Prophet.

### Подготовка данных

In [ ]:
# Заново читаем и готовим данные
df = get_data()

# Количество дней, за которое надо предсказать
N = 60

# Коэффициенты регрессии = коэффициенты для средневзвешенного
coeff = [0.11628278, 0.08466846, 0.12683852, 0.07743247, 0.13885572, 0.10315013, 0.13071674, 0.22205518]

In [ ]:
# Процедура, которая совершает предсказание для одного дня и добавляет предсказание в датасет
def predict_one_day(df,i=1):
    print('Step: ', i )
    
    # Добавляем новую строку с пустым значением, которое будем предсказывать
    df.loc[df.index[-1] + timedelta(days=1),'PAY'] = 0
    
    # Новый стартовый индекс прогноза
    START_DATE = df.index[-1]
    
    # Нормализуем
    scaler = MinMaxScaler()
    df_scal = scaler.fit_transform(df)
    df_scal = pd.DataFrame(df_scal, columns=df.columns)

    # Разделим на тренировочную и тестовую части
    test = df_scal[df.index>=START_DATE]
    train = df_scal[df.index<START_DATE]
    
    # Генерируем набор признаков из предыдущих значений для обучающей выборки
    train_x = []
    for i in range(INPUT_LEN, len(train)):
        train_x.append(train.iloc[i-INPUT_LEN:i])
    train_y = train.iloc[INPUT_LEN:]
    train_x = np.array(train_x)
    train_y = np.array(train_y)
    train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
    
    # Генерируем набор признаков из предыдущих значений для тестовой выборки
    test_x = []
    for i in range(len(df_scal)-1, len(df_scal)):
        test_x.append(df_scal.iloc[i-INPUT_LEN:i])
    test_x = np.array(test_x)
    test_y = [0]
    test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))
    
    # Предсказания разных моделей для одного дня
    predictions = []
    for i in range(len(modelsNN)):
        m = modelsNN[i]
        if i==3 or i==4:
            yhat = m.predict(test_x, batch_size=1)
        else:
            yhat = m.predict(test_x)
        temp = df_scal.iloc[-1].copy()
        temp.PAY = yhat
        temp_df = pd.DataFrame(temp)
        temp_df = temp_df.rename(columns = {temp_df.columns[0]: 'PAY'})
        predictions.append(scaler.inverse_transform(temp_df)[0][0])
    
    # Отдельно предсказываем с помощью Prophet
    train_data_pr = df[:-1].copy()
    train_data_pr = train_data_pr.reset_index()
    train_data_pr.columns = ['ds','y'] 

    model_prop = Prophet(holidays=holiday,         
                 changepoint_prior_scale = 0.001, 
                 weekly_seasonality = True,       
                 yearly_seasonality = True,       
                 n_changepoints = 100             
                )
    model_prop.add_regressor('sunday_date')
    model_prop.add_regressor('bad_date')
    model_prop.add_regressor('good_date')
    model_prop.add_regressor('day_of_year')
    model_prop.add_regressor('day_of_week')

    train_data_pr['day_of_year'] = train_data_pr['ds'].dt.dayofyear
    train_data_pr['day_of_week'] = train_data_pr['ds'].dt.day_of_week
    train_data_pr['sunday_date'] = train_data_pr['ds'].apply(sunday_date)
    train_data_pr['bad_date'] = train_data_pr['ds'].apply(bad_date)
    train_data_pr['good_date'] = train_data_pr['ds'].apply(good_date)

    model_prop.fit(train_data_pr)
    future = model_prop.make_future_dataframe(periods=1)

    future['sunday_date'] = future['ds'].apply(sunday_date)
    future['bad_date'] = future['ds'].apply(bad_date)
    future['good_date'] = future['ds'].apply(good_date)
    future['day_of_year'] = future['ds'].dt.dayofyear
    future['day_of_week'] = future['ds'].dt.day_of_week

    forecast = model_prop.predict(future)

    prediction_prop = forecast[forecast.ds>=START_DATE][['ds','yhat']]
    prediction_prop = prediction_prop.rename(columns = {'ds': 'Date', 'yhat': 'PAY'})
    prediction_prop = prediction_prop.set_index('Date')
    
    prediction_prop['PAY'] = prediction_prop.apply(lambda row: \
                row['PAY']*1.2 if row.name.day>=17 and row.name.day<=25 else \
                row['PAY'] , axis=1)
    
    predictions.append(prediction_prop.iloc[0,0])
    
    # Получаем среднее от всех предсказаний
    new_value = sum(predictions)/len(predictions)
    
    # Получаем средневзвешенное
#     new_value = \
#     coeff[0]*predictions[0] + \
#     coeff[1]*predictions[1] + \
#     coeff[2]*predictions[2] + \
#     coeff[3]*predictions[3] + \
#     coeff[4]*predictions[4] + \
#     coeff[5]*predictions[5] + \
#     coeff[6]*predictions[6] 
    
    # Корректируем срезанные выбросы
    curr_index = df.index[-1]
    if curr_index.day >= 17 and curr_index.day <= 25:
        print(curr_index, new_value)
        new_value *= 1.1
        print(curr_index, new_value)
    
    df.iloc[len(df)-1].PAY = new_value
    return df

### Прогнозирование

In [ ]:
for i in range(1, N+1):
    df = predict_one_day(df,i)

In [ ]:
# Построим график исходных данных и график прогноза на будущее
start_plot = '2020-03-30'

plt.figure(figsize=(25, 12))
plt.plot(df.index[(df.index>LAST_REAL) ], 
         df.loc[(df.index>LAST_REAL), PRED_COLUMN], label="prediction", alpha=.7)
plt.plot(df.index[(df.index>=start_plot) & (df.index<=LAST_REAL)], 
         df.loc[(df.index>=start_plot) & (df.index<=LAST_REAL), PRED_COLUMN], label="real", alpha=.7)
plt.scatter(df.index[(df.index>LAST_REAL)], 
         df.loc[(df.index>LAST_REAL) , PRED_COLUMN], label="prediction", alpha=.7)
plt.scatter(df.index[(df.index>=start_plot) & (df.index<=LAST_REAL)], 
         df.loc[(df.index>=start_plot) & (df.index<=LAST_REAL), PRED_COLUMN], label="real", alpha=.7)
plt.legend() 
plt.title(PRED_COLUMN + " Prediction")
plt.show() 

In [ ]:
# Построим график только прогноза
plt.figure(figsize=(25, 12))
plt.plot(df.index[(df.index>LAST_REAL) ], 
         df.loc[(df.index>LAST_REAL), PRED_COLUMN], label="prediction", alpha=.7)
plt.scatter(df.index[(df.index>LAST_REAL)], 
         df.loc[(df.index>LAST_REAL) , PRED_COLUMN], label="prediction", alpha=.7)
plt.legend() 
plt.title(PRED_COLUMN + " Prediction")
plt.show() 

По графику видно, что прогноз учитывает дни недели и недели в месяце.

### Сохранение результата

In [ ]:
df_res = df[df.index > LAST_REAL]
df_res.to_csv('forecast_' + str(N) + '.csv')

# Заключение

В данной работе рассматривались разные спjсобы предсказания оплаты, ежедневно поступающей от общего объёма должников энергосбытовой компании.      

**Работа состояла из нескольких этапов:**    
1) Чтение и преобразование данных;     
2) Визуализация временного ряда и его сезонных компонентов;    
3) Оценка стационарности ряда и возможности применения тех или иных методов моделирования;    
4) Построение и проверка модели прогнозирования ML-методом SARIMAX;    
5) Построение и проверка прогнозирования нейронными сетями разных архитектур;     
6) Построение и проверка специальной модели Prophet от Facebook для прогнозирования временных рядов с выраженной сезонностью;   
7) Построение и проверка ансамбля из разных моделей;   
8) Выдача прогноза оплаты на заданное количество дней.

**Выводы по применению моделей:**     
1) Временной ряд нестационерен, имеет сложную структуру и плохо прогнозируется классическим метотодом ARIMA, даже в его разновидности SARIMAX, учитывающей сезонность.     
2) Разложение ряда на тренд, сезонность, остаток и последующее прогнозирование методом SARIMAX отдельно каждой компоненты дают существенно лучший результат. Возможно, дополнительное разложение тренда и остатка аналогичным образом дали бы ещё более точную оценку.    
3) Применение нейронных сетей даёт метрику лучше, чем Prophet и SARIMAX.     
4) Среди разных опробованных видов нейронных сетей лучше всего себя зарекомендовала смешанная архитектура: свёрточные слои + рекуррентные, то есть архитектура вида CNN + LSTM.      
5) Модель Prophet неплохо прогнозирует при условии, что нет выбросов. Выбросы данная модель игнорирует и дает плохой прогноз в точках локальных экстремумов.      
6) Применение ансамбля из моделей NN + Prophet существенно улучшает метрику прогноза.     
7) В целом из-за сложности временного ряда очень точные прогнозы (mape<10%) получить не удается. Наилучшая ошбка прогнозирования составляет примерно 17%.